<a href="https://colab.research.google.com/github/dsriwajana/pointcloud/blob/main/rawcode/voxel_keypoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version
!pip install open3d
!pip install trimesh

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
     |████████████████████████████████| 188.4MB 81kB/s 
     |████████████████████████████████| 634kB 13.1MB/s 


In [2]:
import torch
import open3d as o3d
import numpy as np
import trimesh as tm

import plotly.graph_objects as go
import plotly.express as px

In [31]:
# Read .ply file
input_file = "/cloud_bin_0.ply"
pcd = o3d.io.read_point_cloud(input_file) # Read the point cloud

# Downsample
downpcd = pcd.voxel_down_sample(voxel_size=0.05)

# Convert open3d format to numpy array
# Here, you have the point cloud in numpy format. 
pointcloudxyzxyz = np.asarray(downpcd.points) 
pointcloudxxyyzz = np.transpose(pointcloudxyzxyz)

# Check
print(pointcloudxxyyzz.size)
print(pointcloudxyzxyz)
print(pointcloudxxyyzz)

15624
[[-0.58080001 -1.42920001  3.48439999]
 [-0.45165853 -1.31180486  3.44590247]
 [-0.45530136 -1.33861644  3.44534249]
 ...
 [-0.69504347 -0.59686955  2.64939132]
 [-0.57799999  0.27600002  2.26799997]
 [ 0.3990417  -0.59674999  2.65129169]]
[[-0.58080001 -0.45165853 -0.45530136 ... -0.69504347 -0.57799999
   0.3990417 ]
 [-1.42920001 -1.31180486 -1.33861644 ... -0.59686955  0.27600002
  -0.59674999]
 [ 3.48439999  3.44590247  3.44534249 ...  2.64939132  2.26799997
   2.65129169]]


In [95]:
# define function to visualize
def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))
    fig = go.Figure(data=data,
                    layout=go.Layout(
                        updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    y=1,
                                    x=0.8,
                                    xanchor='left',
                                    yanchor='bottom',
                                    pad=dict(t=45, r=10),
                                    buttons=[dict(label='Play',
                                                    method='animate',
                                                    args=[None, dict(frame=dict(duration=50, redraw=True),
                                                                    transition=dict(duration=0),
                                                                    fromcurrent=True,
                                                                    mode='immediate'
                                                                    )]
                                                    )
                                            ]
                                    )
                                ]
                    ),
                    frames=frames
            )

    return fig

def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='red')),
                      selector=dict(mode='markers'))
    fig.show()
        

In [89]:
# visualize test.ply
pcshow(pointcloudxxyyzz[0],pointcloudxxyyzz[1],pointcloudxxyyzz[2])


[Scatter3d({
    'mode': 'markers',
    'x': array([-1.83155002, -1.70240854, -1.70605137, ..., -1.94579349, -1.82875   ,
                -0.85170832]),
    'y': array([-0.48315834, -0.36576319, -0.39257477, ...,  0.34917211,  1.22204169,
                 0.34929168]),
    'z': array([ 0.88660832,  0.8481108 ,  0.84755082, ...,  0.05159966, -0.3297917 ,
                 0.05350002])
})]


----------------------------------Get center and get range is not used in this code atm ---------------------------------------


In [10]:
# voxel size and margin (DONT KNOW HOW TO CHOOSE THE NUMBER)
voxelSize = 0.05; #in meters
voxelMargin = 5; #in margin

# Get center (location) (maybe needed)
objModelpts = downpcd.get_center()
print("location = "+str(objModelpts))

# get range x y z
objModelRangeX = [min(pointcloudxxyyzz[0])-voxelSize*20,max(pointcloudxxyyzz[0])+voxelSize*20]
print("[min x, max x] = "+str(objModelRangeX))
objModelRangeY = [min(pointcloudxxyyzz[1])-voxelSize*20,max(pointcloudxxyyzz[1])+voxelSize*20]
print("[min y, max y] = "+str(objModelRangeY))
objModelRangeZ = [min(pointcloudxxyyzz[2])-voxelSize*20,max(pointcloudxxyyzz[2])+voxelSize*20]
print("[min z, max z] = "+str(objModelRangeZ))

location = [-0.06922717 -0.3381356   2.32930508]
[min x, max x] = [-2.3336842164658664, 2.49399995803833]
[min y, max y] = [-2.43114287512643, 1.6864000320434571]
[min z, max z] = [-0.19557894217340566, 4.48628568649292]


In [123]:
# Get keypoint (random for now)
print('Point cloud currently contains '+str(pointcloudxyzxyz.shape[0])+' points')
import random as r
keypointidx= r.randint(0,pointcloudxyzxyz.shape[0])
keypoint = pointcloudxyzxyz[keypointidx]
print("keypoint location is "+ str(keypoint))

# center at keypoint
pcdAtKeypoint = downpcd
pcdAtKeypoint.translate(keypoint*-1, relative=True)  # translate by negative position of keypoints (use True) 
pointcloudxxyyzzAtKeypoint = np.transpose(pointcloudxyzxyzAtKeypoint)
print(pointcloudxyzxyzAtKeypoint.shape)
# pcshow(pointcloudxxyyzzAtKeypoint[0],pointcloudxxyyzzAtKeypoint[1],pointcloudxxyyzzAtKeypoint[2]) # for visualization

# compute voxel grid
[gridx, gridy, gridz] = np.meshgrid(np.arange(voxelSize*-5,voxelSize*5, voxelSize),
                                    np.arange(voxelSize*-5,voxelSize*5, voxelSize),
                                    np.arange(voxelSize*-5,voxelSize*5, voxelSize))

# build kdtree
from scipy.spatial import KDTree

pointcloudxyzxyzRemovedOutOfBound = pointcloudxyzxyzAtKeypoint
for i in range(pointcloudxyzxyzRemovedOutOfBound.shape[0]-1,-1,-1):
  if np.amax(pointcloudxyzxyzRemovedOutOfBound[i])>voxelSize*10:
    pointcloudxyzxyzRemovedOutOfBound = np.delete(pointcloudxyzxyzRemovedOutOfBound,i,axis=0)

print(pointcloudxyzxyzRemovedOutOfBound.shape)
modelKDT = KDTree(pointcloudxyzxyzRemovedOutOfBound)

# voxel for KDTree
voxelxxyyzz = np.array([gridx.reshape(-1), gridy.reshape(-1), gridz.reshape(-1)])

# array storing distance and index
nnarraydistance = np.zeros(gridx.reshape(-1).size)
nnarrayindex = np.zeros(gridx.reshape(-1).size)

# NN search
for i in range(gridx.reshape(-1).size): #gridx.reshape(-1).size
  dd, ii = modelKDT.query(np.array([gridx.reshape(-1)[i], gridy.reshape(-1)[i], gridz.reshape(-1)[i]]),k=1)
  nnarraydistance[i] = dd 
  nnarrayindex[i] = ii  
# print(nnarraydistance)
# print(nnarrayindex)

# Calculate TDF around the keypoints
voxelGridTDF = [1.0 - min( i/(voxelSize*voxelMargin) ,1.0) for i in nnarraydistance];
print(voxelGridTDF)

Point cloud currently contains 5208 points
keypoint location is [ 0.90096002 -1.61171995  1.31016005]
(5208, 3)
(1277, 3)
[0.0, 0.06276972467308628, 0.20200220797653623, 0.33433581951229185, 0.43111292541275203, 0.513621687438343, 0.5974484897755756, 0.6284520815385095, 0.6593182630245358, 0.6957899068704654, 0.03650672557817025, 0.16746207310454586, 0.30142972633657106, 0.4377482831250391, 0.5565468507531983, 0.6448044314437127, 0.7525170419719046, 0.8070027154506119, 0.8423167147640592, 0.876841666386893, 0.1372605323516095, 0.286473897533167, 0.40508857065543724, 0.5013806653122943, 0.6131293659097719, 0.690542537796027, 0.8130134258907294, 0.871750056632338, 0.8831069250255658, 0.8835181781941538, 0.19980034477615705, 0.3635099387393089, 0.5000802985009296, 0.582005002649313, 0.648672218029228, 0.6702740651455543, 0.7092010868506884, 0.7249579230296216, 0.7128346060000295, 0.6967376474336776, 0.2149397555585394, 0.3826510733258158, 0.5246897780090897, 0.6117733690325589, 0.64590247

In [51]:
# # ------------------------------- point cloud to TDF --------------------------------------- ctrl + / to comment/uncomment

# # compute voxel grid
# [gridx, gridy, gridz] = np.meshgrid(np.arange(objModelRangeX[0],objModelRangeX[1], voxelSize),
#                                     np.arange(objModelRangeY[0],objModelRangeY[1], voxelSize),
#                                     np.arange(objModelRangeZ[0],objModelRangeZ[1], voxelSize))

# #build kdtree
# print("build kdtree")
# from scipy.spatial import KDTree
# modelKDT = KDTree(pointcloudxyzxyz)
# #print(gridx.reshape(-1).shape)
# #print(gridy.reshape(-1).shape)
# #print(gridz.reshape(-1).shape)

# # voxel for KDTree
# voxelxxyyzz = np.array([gridx.reshape(-1), gridy.reshape(-1), gridz.reshape(-1)])

# # array storing distance and index
# nnarraydistance = np.zeros(gridx.reshape(-1).size)
# nnarrayindex = np.zeros(gridx.reshape(-1).size)

# # NN search
# for i in range(gridx.reshape(-1).size): #gridx.reshape(-1).size
#   dd, ii = modelKDT.query(np.array([gridx.reshape(-1)[i], gridy.reshape(-1)[i], gridz.reshape(-1)[i]]),k=1)
#   nnarraydistance[i] = dd 
#   nnarrayindex[i] = ii  
# print(nnarraydistance)
# print(nnarrayindex)

# # Calculate TDF
# voxelGridTDF = [1.0 - min( i/(voxelSize*voxelMargin) ,1.0) for i in nnarraydistance];
# # print(voxelGridTDF) # long processing time and may make your PC frozen

In [115]:
# pointcloudtest = np.array([[1,1,1],[2,2,2],[1,1,1],[0,0,0]])
# pointcloudtest.shape[0]
# for i in range(pointcloudtest.shape[0]-1,-1,-1):
#   print(i)
#   if np.amax(pointcloudtest[i])>1.2:
#     print("remove "+str(pointcloudtest[i]))
#     pointcloudtest = np.delete(pointcloudtest,i,axis=0)
# print(pointcloudtest)

3
2
1
remove [2 2 2]
0
[[1 1 1]
 [1 1 1]
 [0 0 0]]
